In [2]:
from pythreejs import MeshPhongMaterial, ShaderMaterial, ShaderLib, ShaderChunk

In [5]:
class CustomMaterial(ShaderMaterial):

    def __init__(self, typ):
        self.types = {
            'diffuse': 'c',
            'uvTransform': 'm3',
            'normalScale': 'v2',
            'fogColor': 'c',
            'emissive': 'c'}

        shader = ShaderLib[typ]
        
        fragmentShader = """
        uniform float alpha;
        """
        frag_from = "gl_FragColor = vec4( outgoingLight, diffuseColor.a );"
        frag_to = """
            if ( gl_FrontFacing ) {
                gl_FragColor = vec4( outgoingLight, alpha * diffuseColor.a );
            } else {
                gl_FragColor = vec4(diffuseColor.r, diffuseColor.g, diffuseColor.b, alpha * diffuseColor.a);
            }"""
        fragmentShader += shader["fragmentShader"].replace(frag_from, frag_to)

        vertexShader=shader["vertexShader"]
        uniforms=shader["uniforms"]
        uniforms["clip_alpha"] = dict(value=0.6)

        super().__init__(uniforms=uniforms, vertexShader=vertexShader, fragmentShader=fragmentShader)
        self.alpha = 1.0
        self.lights = True

    @property
    def color(self):
        return self.uniforms["diffuse"]["value"]

    @color.setter
    def color(self, value):
        self.update("diffuse", value)
        
    @property
    def alpha(self):
        return self.uniforms["alpha"]["value"]

    @alpha.setter
    def alpha(self, value):
        self.update("alpha", value)
        
    def update(self, key, value):
        uniforms = dict(**self.uniforms)
        if self.types.get(key) is None:
            uniforms[key] = {'value': value}
        else:
            uniforms[key] = {'type': self.types.get(key), 'value': value}
        self.uniforms = uniforms
        self.needsUpdate = True


In [6]:
from jupyter_cadquery import display, Assembly, Part, CadqueryDisplay
import cadquery as cq

xy = cq.Workplane("XY")

d = 2
box1 = xy.box(10,20,30)
box2 = xy.box(10-d, 20-d, 30-d)
p = Part(box1.cut(box2), color=(1,0,0))
d = CadqueryDisplay()
d.display(Assembly([p]))

shell = d.cq_view.pickable_objects.children[0]

In [8]:
from pythreejs import *
from IPython.display import display

from ipywidgets import Layout


material = CustomMaterial("standard")
material.side = "DoubleSide"
material.alpha = 0.7
material.opacity = 1.0
material.transparent = True
material.clipping = True
material.color = "#ff0000"

obj = Mesh(
    TorusKnotBufferGeometry(radius=20, tube=5, tubularSegments=64, radialSegments=64, p=2, q=3), 
    material=material
)

obj = Mesh(shell.geometry, material=material)
key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.4)

width = 300
height = 300

c = PerspectiveCamera(position=[30, 30, 30], up=[0, 0, 1])
c.add(key_light)

scene = Scene(children=[obj, c, AmbientLight(color='#888')], background=None)

renderer = Renderer(camera=c, 
                    scene=scene,
                    alpha=True,
                    clearOpacity=0,
                    controls=[OrbitControls(controlling=c)],
                    width=width, height=height)

renderer.localClippingEnabled = True;
renderer.clippingPlanes = [] # [Plane((0,-1,0), 2.0)] # , Plane((0,0,-1), 2.0)]
display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.4, position=(3.0, 5.0,…

In [9]:
material.update("metalness", 0.0)

In [76]:
import time
for i in range(-15, 5):
    renderer.clippingPlanes = [Plane((0,-1,0), i/2.0), Plane((0,0,-1), i/2.0)]
    time.sleep(0.05)

In [84]:
material.update("opacity", 1.0)
material.update("transparent", True)

In [327]:
print(material.fragmentShader)


        uniform float alpha;
        #define PHYSICAL
uniform vec3 diffuse;
uniform vec3 emissive;
uniform float roughness;
uniform float metalness;
uniform float opacity;
#ifndef STANDARD
	uniform float clearCoat;
	uniform float clearCoatRoughness;
#endif
varying vec3 vViewPosition;
#ifndef FLAT_SHADED
	varying vec3 vNormal;
#endif
#include <common>
#include <packing>
#include <dithering_pars_fragment>
#include <color_pars_fragment>
#include <uv_pars_fragment>
#include <uv2_pars_fragment>
#include <map_pars_fragment>
#include <alphamap_pars_fragment>
#include <aomap_pars_fragment>
#include <lightmap_pars_fragment>
#include <emissivemap_pars_fragment>
#include <bsdfs>
#include <cube_uv_reflection_fragment>
#include <envmap_pars_fragment>
#include <envmap_physical_pars_fragment>
#include <fog_pars_fragment>
#include <lights_pars_begin>
#include <lights_physical_pars_fragment>
#include <shadowmap_pars_fragment>
#include <bumpmap_pars_fragment>
#include <normalmap_pars_fragment>
#include

# Scratch

In [99]:
ShaderLib["standard"]["uniforms"]

{'diffuse': {'type': 'c', 'value': '#eeeeee'},
 'opacity': {'value': 1},
 'map': {'value': None},
 'uvTransform': {'type': 'm3', 'value': [1, 0, 0, 0, 1, 0, 0, 0, 1]},
 'alphaMap': {'value': None},
 'envMap': {'value': None},
 'flipEnvMap': {'value': -1},
 'reflectivity': {'value': 1},
 'refractionRatio': {'value': 0.98},
 'maxMipLevel': {'value': 0},
 'aoMap': {'value': None},
 'aoMapIntensity': {'value': 1},
 'lightMap': {'value': None},
 'lightMapIntensity': {'value': 1},
 'emissiveMap': {'value': None},
 'bumpMap': {'value': None},
 'bumpScale': {'value': 1},
 'normalMap': {'value': None},
 'normalScale': {'type': 'v2', 'value': [1, 1]},
 'displacementMap': {'value': None},
 'displacementScale': {'value': 1},
 'displacementBias': {'value': 0},
 'roughnessMap': {'value': None},
 'metalnessMap': {'value': None},
 'fogDensity': {'value': 0.00025},
 'fogNear': {'value': 1},
 'fogFar': {'value': 2000},
 'fogColor': {'type': 'c', 'value': '#ffffff'},
 'ambientLightColor': {'value': []},


In [105]:
for k,v in ShaderLib["standard"]["uniforms"].items():
    try:
        print(k, material.__getattribute__(k))
    except:
        print(k, "missing")

diffuse missing
opacity 0.6
map None
uvTransform missing
alphaMap None
envMap None
flipEnvMap missing
reflectivity missing
refractionRatio 0.98
maxMipLevel missing
aoMap None
aoMapIntensity 1.0
lightMap None
lightMapIntensity 1.0
emissiveMap None
bumpMap None
bumpScale 1.0
normalMap None
normalScale (1.0, 1.0)
displacementMap None
displacementScale 1.0
displacementBias 0.0
roughnessMap None
metalnessMap None
fogDensity missing
fogNear missing
fogFar missing
fogColor missing
ambientLightColor missing
directionalLights missing
directionalShadowMap missing
directionalShadowMatrix missing
spotLights missing
spotShadowMap missing
spotShadowMatrix missing
pointLights missing
pointShadowMap missing
pointShadowMatrix missing
hemisphereLights missing
rectAreaLights missing
emissive #000000
roughness 0.5
metalness 0.5
envMapIntensity 1.0
clip_alpha missing


In [41]:
@register
class MeshPhongMaterial2(ShaderMaterial):

    _model_name = Unicode('MeshPhongMaterialModel').tag(sync=True)

    alphaMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    aoMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    aoMapIntensity = CFloat(1, allow_none=False).tag(sync=True)
    bumpMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    bumpScale = CFloat(1, allow_none=False).tag(sync=True)
    color = Color("#ffffff", allow_none=False).tag(sync=True)
    combine = Enum(Operations, "MultiplyOperation", allow_none=False).tag(sync=True)
    displacementMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    displacementScale = CFloat(1, allow_none=False).tag(sync=True)
    displacementBias = CFloat(0, allow_none=False).tag(sync=True)
    emissive = Color("#000000", allow_none=False).tag(sync=True)
    emissiveMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    emissiveIntensity = CFloat(1, allow_none=False).tag(sync=True)
    envMap = Instance(CubeTexture, allow_none=True).tag(sync=True, **widget_serialization)
    lightMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    lightMapIntensity = CFloat(1, allow_none=False).tag(sync=True)
    map = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    morphNormals = Bool(False, allow_none=False).tag(sync=True)
    morphTargets = Bool(False, allow_none=False).tag(sync=True)
    normalMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    normalScale = Vector2(default_value=[1, 1]).tag(sync=True)
    reflectivity = CFloat(1, allow_none=False).tag(sync=True)
    refractionRatio = CFloat(0.98, allow_none=False).tag(sync=True)
    shininess = CFloat(30, allow_none=False).tag(sync=True)
    skinning = Bool(False, allow_none=False).tag(sync=True)
    specular = Color("#111111", allow_none=False).tag(sync=True)
    specularMap = Instance(Texture, allow_none=True).tag(sync=True, **widget_serialization)
    wireframe = Bool(False, allow_none=False).tag(sync=True)
    wireframeLinewidth = CFloat(1, allow_none=False).tag(sync=True)
    wireframeLinecap = Unicode("round", allow_none=False).tag(sync=True)
    wireframeLinejoin = Unicode("round", allow_none=False).tag(sync=True)
    type = Unicode("MeshPhongMaterial2", allow_none=False).tag(sync=True)


    def __init__(self, **kwargs):
        phongShader = ShaderLib['phong']
        frag_from = "gl_FragColor = vec4( outgoingLight, diffuseColor.a );"
        frag_to = """
        console.log(gl_FrontFacing)
        if ( gl_FrontFacing ) {
            gl_FragColor = vec4( outgoingLight, diffuseColor.a );
        } else {
            gl_FragColor = diffuseColor;
        }"""
        frag = phongShader["fragmentShader"].replace(frag_from, frag_to)
        super().__init__(
            uniforms=phongShader["uniforms"],
            vertexShader=phongShader["vertexShader"],
            fragmentShader=frag,
            **kwargs
        )
        self.lights = True

if six.PY3:
    import inspect
    MeshPhongMaterial2.__signature__ = inspect.signature(MeshPhongMaterial2.__init__)


In [1]:
from pythreejs import *
class SectionHelper(Mesh):
    def __init__(self, object, hexOrMaterial):
        if isinstance(hexOrMaterial, MeshBasicMaterial):
            material = hexOrMaterial
        else:
            color = hexOrMaterial
            material = MeshBasicMaterial(color=color, side="BackSide")

        super().__init__(object.geometry, material)

        self.matrix = object.matrixWorld
        self.matrixAutoUpdate = False

In [4]:
from ipywidgets import Layout
import time

# material = MeshPhongMaterial(color = "green", transparent=False, opacity=0.8, side="FrontSide")
material = CustomMaterial("standard")
material.color = "#ff0000"
material.clipping = True
material.side = "DoubleSide"

torus = Mesh(
    TorusKnotBufferGeometry(radius=20, tube=5, tubularSegments=64, radialSegments=64, p=2, q=3), 
    material=material
)

key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)

c = PerspectiveCamera(position=[0, 100, 100], up=[0, 1, 0], children=[key_light])

scene = Scene(children=[torus, c, AmbientLight(color='#777777')], background=None)

renderer = Renderer(camera=c, 
                    scene=scene,
                    alpha=True,
                    clearOpacity=0,
                    controls=[OrbitControls(controlling=c)],
                    width=500, height=500)

renderer.localClippingEnabled = True;
display(renderer)

for i in range(-75, 75):
    renderer.clippingPlanes = [Plane((0,-1,0), i/2.0)]
    time.sleep(0.05)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

In [133]:
cm.material.uniforms

{'diffuse': {'type': 'c', 'value': '#ff00ff'},
 'opacity': {'value': 1},
 'map': {'value': None},
 'uvTransform': {'type': 'm3', 'value': [1, 0, 0, 0, 1, 0, 0, 0, 1]},
 'alphaMap': {'value': None},
 'envMap': {'value': None},
 'flipEnvMap': {'value': -1},
 'reflectivity': {'value': 1},
 'refractionRatio': {'value': 0.98},
 'maxMipLevel': {'value': 0},
 'aoMap': {'value': None},
 'aoMapIntensity': {'value': 1},
 'lightMap': {'value': None},
 'lightMapIntensity': {'value': 1},
 'emissiveMap': {'value': None},
 'bumpMap': {'value': None},
 'bumpScale': {'value': 1},
 'normalMap': {'value': None},
 'normalScale': {'type': 'v2', 'value': [1, 1]},
 'displacementMap': {'value': None},
 'displacementScale': {'value': 1},
 'displacementBias': {'value': 0},
 'roughnessMap': {'value': None},
 'metalnessMap': {'value': None},
 'fogDensity': {'value': 0.00025},
 'fogNear': {'value': 1},
 'fogFar': {'value': 2000},
 'fogColor': {'type': 'c', 'value': '#ffffff'},
 'ambientLightColor': {'value': []},


In [67]:

from pythreejs import *
import ipywidgets as widgets
from IPython.display import display

In [68]:

vertex_shader = """
uniform float time;
uniform vec2 resolution;

void main() {
    vec3 pos = vec3(position.x + time * resolution.x, position.y + time * resolution.y, position.z);
    gl_Position = projectionMatrix * modelViewMatrix * vec4( pos, 1.0 );
}
"""

In [69]:
fragment_shader = """
uniform vec3 user_color;

void main() {
    #ifdef FIX_COLOR
    gl_FragColor = vec4(0.5, 0, 0, 1.0);
    #else
    gl_FragColor = vec4(user_color, 1.0);
    #endif
}
"""

In [70]:
material = ShaderMaterial(
    uniforms=dict(
        time=dict(value=0.0),
        resolution=dict(value=(1, 1)),
        user_color=dict(value='green'),
        **UniformsLib['common']
    ),
    defines=dict(
        FIX_COLOR=1,
    ),
    vertexShader=vertex_shader,
    fragmentShader=fragment_shader,
)

In [71]:
material

Preview(child=ShaderMaterial(defines={'FIX_COLOR': 1}, fragmentShader='\nuniform vec3 user_color;\n\nvoid main…

In [72]:
material.defines = dict()
material.needsUpdate = True

In [74]:

# Uniform editors:
picker = widgets.ColorPicker(value=material.uniforms['user_color']['value'])
time_slider = widgets.FloatSlider(value=material.uniforms['time']['value'], min=-15, max=15)

# Interactive code:
def update_uniforms(change):
    uniforms = dict(**material.uniforms)
    print(dict(value=time_slider.value))
    uniforms.update(
        time=dict(value=time_slider.value),
        user_color=dict(value=picker.value),
    )
    material.uniforms = uniforms
    material.needsUpdate = True

picker.observe(update_uniforms)
time_slider.observe(update_uniforms)

# Present "dashboard" in VBox:
widgets.VBox([
    Preview(material),
    picker,
    time_slider,
])

In [75]:
print(dict(value=time_slider.value))

{'value': -2.5}


In [76]:
dict(**material.uniforms)

{'time': {'value': 0.4},
 'resolution': {'value': [1, 1]},
 'user_color': {'value': '#d92a4f'},
 'diffuse': {'type': 'c', 'value': '#eeeeee'},
 'opacity': {'value': 1},
 'map': {'value': None},
 'uvTransform': {'type': 'm3', 'value': [1, 0, 0, 0, 1, 0, 0, 0, 1]},
 'alphaMap': {'value': None}}

In [78]:
dict(**UniformsLib['common'])

{'diffuse': {'type': 'c', 'value': '#eeeeee'},
 'opacity': {'value': 1},
 'map': {'value': None},
 'uvTransform': {'type': 'm3', 'value': [1, 0, 0, 0, 1, 0, 0, 0, 1]},
 'alphaMap': {'value': None}}